In [7]:
import cassio
from datasets import load_dataset
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.cassandra import Cassandra
from PyPDF2 import PdfReader
from typing_extensions import Concatenate
import os

In [2]:
load_dotenv()

True

In [3]:
pdf_reader = PdfReader("documents/Anayasa.pdf")

raw_text = ""

for i, page in enumerate(pdf_reader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

print(raw_text)

 
1 
  
 
 
 
 
TÜRKİYE CUMHURİYETİ ANAYASASI 1 2 3 4 
Kanun Numarası  : 2709  
Kabul Tarihi  : 18/10/1982  
Yayımlandığı Resmî Gazete  : Tarih  : 9/11/1982   Sayı   : 17863 (Mükerrer)   
Yayımlandığı Düstur  : Tertip : 5   Cilt : 22  Sayfa :  3 
 
 
BAŞLANGI Ç 5 
 
Türk Vatanı ve M illetinin ebedi varlığını ve Yüce Türk Devletinin bölünmez bütünlüğünü 
belirleyen bu Anayasa, Türkiye Cumhuriyetinin kurucusu, ölümsüz önder ve eşsiz kahraman 
Atatürk’ün belirlediği milliyetçilik anlayışı ve onun inkılap ve ilkeleri doğrultusunda;  
Dünya mi lletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin 
ebedi varlığı, refahı, maddi ve manevi mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi 
yönünde;  
Millet iradesinin mutlak üstünlüğü, egemenliğin kayıtsız şartsız Türk  Milletine ait olduğu ve 
bunu millet adına kullanmaya yetkili kılınan hiçbir kişi ve kuruluşun, bu Anayasada gösterilen 
hürriyetçi demokrasi ve bunun icaplarıyla belirlenmiş hukuk düzeni dışın

In [5]:
cassio.init(token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"), database_id=os.getenv("ASTRA_DB_ID"))

In [8]:
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"))
embedding = OpenAIEmbeddings(openai_api_key = os.getenv("OPENAI_API_KEY"))

In [9]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_demo",
    session=None,
    keyspace=None,
)

In [10]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [12]:
texts[:50]

['1 \n  \n \n \n \n \nTÜRKİYE CUMHURİYETİ ANAYASASI 1 2 3 4 \nKanun Numarası  : 2709  \nKabul Tarihi  : 18/10/1982  \nYayımlandığı Resmî Gazete  : Tarih  : 9/11/1982   Sayı   : 17863 (Mükerrer)   \nYayımlandığı Düstur  : Tertip : 5   Cilt : 22  Sayfa :  3 \n \n \nBAŞLANGI Ç 5 \n \nTürk Vatanı ve M illetinin ebedi varlığını ve Yüce Türk Devletinin bölünmez bütünlüğünü \nbelirleyen bu Anayasa, Türkiye Cumhuriyetinin kurucusu, ölümsüz önder ve eşsiz kahraman \nAtatürk’ün belirlediği milliyetçilik anlayışı ve onun inkılap ve ilkeleri doğrultusunda;  \nDünya mi lletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin \nebedi varlığı, refahı, maddi ve manevi mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi \nyönünde;',
 'Dünya mi lletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin \nebedi varlığı, refahı, maddi ve manevi mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi \nyönünde;  \nMillet iradesinin mutlak üstünlüğü, ege

In [13]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


Run the QA Cycle

Simply run the cells and ask a question -- or quit to stop. (you can also stop execution with the "▪" button on the top toolbar)

In [14]:
first_question = True

while True:
    if first_question:
        query_text = input('\nEnter your question (or type "quit" to exit): ').strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()
    
    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue
    
    first_question = False
    
    print("\nQuestion: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Question: "Anayasada kaç madde var"
ANSWER: "4"

FIRST DOCUMENTS BY RELEVANCE:
    [0.9034] "Madde 4  – Anayasanın 1 inci maddesindeki Devletin şeklinin Cumhuriyet olduğu hakkın ..."
    [0.9033] "VII. Yasama yetkisi  
Madde 7  – Yasama yetkisi Türk Milleti adına Türkiye Büyük Mil ..."
    [0.9033] "Madde 2  – Türkiye Cumhuriyeti, toplumun huzuru, milli dayanışma ve adalet anlayışı  ..."
    [0.9030] "1 
  
 
 
 
 
TÜRKİYE CUMHURİYETİ ANAYASASI 1 2 3 4 
Kanun Numarası  : 2709  
Kabul  ..."

Question: "Anayasa değiştirilebilir mi"
ANSWER: "Anayasa değiştirilemez. Bu konuda Anayasanın 1, 2, 3 ve 4. maddelerinde belirtilen hükümler değiştirilemez ve değiştirilmesi teklif edilemez."

FIRST DOCUMENTS BY RELEVANCE:
    [0.9249] "Madde 4  – Anayasanın 1 inci maddesindeki Devletin şeklinin Cumhuriyet olduğu hakkın ..."
    [0.9221] "sunularak kabul edilmiş, buna ilişkin 22/9/2010 tarihli ve 846 sayılı Yüksek Seçim K ..."
    [0.9199] "Madde 2  – Türkiye Cumhuriyeti, toplumun huzuru, milli day